In [ ]:
# We will use this script to classify traffic signs from the German Traffic Sign Recognition Benchmark using Keras
# and Convolutional Neural Networks. Our CNN model is saved as a sign_model.keras file which we can access.
#
# A folder called clips contains 10, 30 second long clips of driving footage. We will use our model to classify

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from IPython.display import display, Image

In [ ]:
# This uses a a pretrained cascade model that also didnt work because its trained for face detection and isn't really able to detect traffic signs

# Load the trained model
model = load_model('sign_model.keras')  # Replace with the path to your saved model

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the video capture
cap = cv2.VideoCapture('clips/clip_1.mp4')  # Replace with the path to your video

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Preprocess the frame (resize, normalization, etc.)
    # ...
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_color = frame[y:y + h, x:x + w]
        resized_roi = cv2.resize(roi_color, (32, 32))
        preprocessed_roi = resized_roi / 255.0

        # model inference
        input_data = np.expand_dims(preprocessed_roi, axis=0)
        predictions = model.predict(input_data)
        predicted_class = np.argmax(predictions)

        class_label = f"Class: {predicted_class}"
        confidence = f"Confidence: {predictions[0, predicted_class]:.2f}"
        color = (0, 255, 0)  # Green color for the bounding box
        cv2.rectangle(frame, (x,y), (x+w, y+h), color, 2)
        cv2.putText(frame, class_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)
        cv2.putText(frame, confidence, (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Live Detection', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Also sliding window technique, this doesn't work either

model = load_model('sign_model.keras')  # Replace with the path to your saved model

# Define the sliding window size and stride
window_size = (64, 64)
stride = 16

# Open the video capture
cap = cv2.VideoCapture('clips/clip_1.mp4')  # Replace with the path to your video

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Initialize the coordinates of the top-left corner of the sliding window
    x, y = 0, 0

    # Iterate over the frame using the sliding window
    while y + window_size[1] <= frame.shape[0]:
        while x + window_size[0] <= frame.shape[1]:
            # Extract the region of interest (ROI) using the sliding window
            roi = frame[y:y + window_size[1], x:x + window_size[0]]

            # Preprocess the ROI (resize, normalization, etc.)
            # ...
            resized_roi = cv2.resize(roi, (32, 32))
            preprocessed_roi = resized_roi / 255.0

            # Reshape the input data to have the correct shape
            input_data = np.expand_dims(preprocessed_roi, axis=-1)
            input_data = np.expand_dims(input_data, axis=0)

            # Model inference
            predictions = model.predict(input_data)
            predicted_class = np.argmax(predictions)

            # If the predicted class is a road sign, draw a bounding box
            if predicted_class >= 1 and predictions[0, predicted_class] >= 0.75:
                color = (0, 255, 0)  # Green color for the bounding box
                cv2.rectangle(frame, (x, y), (x + window_size[0], y + window_size[1]), color, 2)

            # Move the sliding window horizontally
            x += stride

        # Move the sliding window vertically and reset the horizontal position
        y += stride
        x = 0
        
        cv2.imshow('Object Detection', frame)
        cv2.waitKey(1)
    # Display the frame with bounding boxes

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1000) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
# Technique num3 - Sliding window technique
# Sliding window technique - this doesn't work and isn't a good approach because it uses crazy resources

import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model('sign_model.keras')  # Replace with the path to your saved model

video_path = "clips/clip_1.mp4"  # Replace with the path to your video

cap = cv2.VideoCapture(video_path)

window_size = (64, 64)
step_size = 20

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    for y in range(0, frame.shape[0] - window_size[1], step_size):
        for x in range(0, frame.shape[1] - window_size[0], step_size):
            patch = frame[y:y + window_size[1], x:x + window_size[0]]
            
            resized_patch = cv2.resize(patch, (32, 32))

            prediction = model.predict(np.expand_dims(resized_patch, axis=0))
            confidence = prediction.max()

            if confidence > 0.90:
                cv2.rectangle(frame, (x, y), (x + window_size[0], y + window_size[1]), (0, 255, 0), 2)
        
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()